# Test env

In [1]:

#imports
import os
import streamlit as st
from PIL import Image
from loguru import logger
# local imports
from ingest.ingester import Ingester
from query.querier import Querier
from summarize.summarizer import Summarizer
import settings
import utils as ut
from query.querier import EnumMode
from ingest.ingester import IngestionMode


/opt/homebrew/lib/python3.11/site-packages/pypdf/_crypt_providers/_cryptography.py:32: CryptographyDeprecationWarning: ARC4 has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.ARC4 and will be removed from this module in 48.0.0.
  from cryptography.hazmat.primitives.ciphers.algorithms import AES, ARC4


In [2]:
SYSTEM_PROMPT= """
### OBJECTIVE ###
Je bent een assistent voor de rijksoverheid. Jouw taak is om vragen te beantwoorden in het Nederlands. Zorg ervoor dat je alleen antwoord geeft op basis van de beschikbare context en dat je daar ook naar verwijst in je antwoord.

### AUDIENCE ###
De doelgroep van jouw antwoorden zijn ambtenaren. Geef alle relevante informatie uit de context, antwoord in het Nederlands leg in maximaal 100 woorden zoveel mogelijk uit.

### GUARDRAILS ###
Indien de context onvoldoende informatie bevat om de vraag te beantwoorden, verzin dan geen informatie maar geef aan dat er onvoldoende informatie beschikbaar is.

### INSTRUCTIONS ###
- Beantwoord de vraag altijd in het Nederlands, zelfs als de context in het Engels is gesteld.
- Vermijd het herhalen van de vraag in het antwoord en het herhalen van de instructies. Voer de instructies uit en geef een concreet antwoord op de gestelde vraag.
- Geef een stapsgewijze redenering bij het beantwoorden van de vraag en refereer naar specifieke zinnen uit de context die hebben bijgedragen aan het antwoord.
- Houd je antwoord nauw verbonden met de context en vermijd het toevoegen van informatie die niet expliciet in de context wordt vermeld.

- Voor meer informatie over de context, zeg het bestandsnaam die gevonden is in de source_document. Mits deze beschikbaar is.

### QUESTION ### \n
"""

In [3]:
# Init
querier = Querier()
folderSelected = "kamerVragen"
my_folder_path_selected, my_vectordb_folder_path_selected = ut.create_vectordb_name(folderSelected)
ingester = Ingester(folderSelected, my_folder_path_selected, my_vectordb_folder_path_selected)
ingester.ingest(mode=IngestionMode.question_answer_per_page, forceRebuild=False, addedMetaDataURLCSV="docs/metadata.csv")
querier.make_chain(folderSelected, my_vectordb_folder_path_selected)

/opt/homebrew/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-09-23 17:13:36.061 | INFO     | utils:getEmbeddings:91 - Loaded local embeddings: textgain/allnli-GroNLP-bert-base-dutch-cased
2024-09-23 17:13:36.062 | INFO     | llm_class.llm_class:__init__:38 - Use Local LLM
2024-09-23 17:13:36.062 | INFO     | llm_class.llm_class:__init__:39 - Retrieving mistral-openorca
2024-09-23 17:13:36.063 | INFO     | llm_class.llm_class:__init__:41 - Using local api url http://127.0.0.1:11434
2024-09-23 17:13:36.064 | INFO     | llm_class.llm_class:__init__:52 - Retrieved mistral-openorca
2024-09-23 17:13:37.221 | INFO     | utils:getEmbeddings:91 - Loaded local embeddings: textgain/allnli-GroNLP-bert-base-dutch-cased
2024-09-23 17:13:37.241 | INFO     | ingest.ingester:ingest:134 - Skipping ingestion of file

In [4]:
querier.ask_question("""
                     Kunt u aangeven welke maatregelen en acties ondernomen zijn naar
aanleiding van het inspectierapport, welke tekortkomingen nog niet verholpen
zijn en op welke termijn dat wel wordt gedaan?
                     """, mode=EnumMode.metadata, system_prompt_override=SYSTEM_PROMPT)

2024-09-23 17:13:38.173 | INFO     | query.querier:get_documents_with_scores:126 - Topscore most similar docs: 0.7394572496414185


Documents: {'Source': 'https://gegevensmagazijn.tweedekamer.nl/OData/v4/2.0/Document(18e8776a-9d24-4977-bc5f-00eb5d5ba5f7)/resource', 'Subject': 'Antwoord op vragen van het lid Peters over het bericht van een “veroordeelde stalker, neptherapeut, fraudeur en toch jeugdzorgdirecteur”', 'author': '', 'chunk': 1, 'filename': '18e8776a-9d24-4977-bc5f-00eb5d5ba5f7.pdf', 'page_number': 7, 'source': 'p7-1', 'title': 'None'}
 Om de vragen te beantwoorden, kunt u zich richten aan de beschikbare context. Als er geen informatie in de context is die overeenkomt met de gestelde vraag, zeg je dat er geen informatie beschikbaar is om de vraag te beantwoorden. 

Voorbeeld: Vraag: Kunt u aangeven welke maatregelen en acties ondernomen zijn naar aanleiding van het inspectierapport, welke tekortkomingen nog niet verholen zijn en op welke termijn dat wel wordt gedaan? Antwoord: Er is geen informatie in de context over de genomen maatregelen, acties of tekortkomingen naar aanleiding van het inspectierapport

2024-09-23 17:13:53.097 | INFO     | query.querier:ask_question:187 - Topscore: 0.7394572496414185


{'question': '\n### OBJECTIVE ###\nJe bent een assistent voor de rijksoverheid. Jouw taak is om vragen te beantwoorden in het Nederlands. Zorg ervoor dat je alleen antwoord geeft op basis van de beschikbare context en dat je daar ook naar verwijst in je antwoord.\n\n### AUDIENCE ###\nDe doelgroep van jouw antwoorden zijn ambtenaren. Geef alle relevante informatie uit de context, antwoord in het Nederlands leg in maximaal 100 woorden zoveel mogelijk uit.\n\n### GUARDRAILS ###\nIndien de context onvoldoende informatie bevat om de vraag te beantwoorden, verzin dan geen informatie maar geef aan dat er onvoldoende informatie beschikbaar is.\n\n### INSTRUCTIONS ###\n- Beantwoord de vraag altijd in het Nederlands, zelfs als de context in het Engels is gesteld.\n- Vermijd het herhalen van de vraag in het antwoord en het herhalen van de instructies. Voer de instructies uit en geef een concreet antwoord op de gestelde vraag.\n- Geef een stapsgewijze redenering bij het beantwoorden van de vraag e

In [5]:
querier.get_documents_with_scores("""Hoe wordt er gekeken naar het platform X""")

2024-09-23 17:13:53.228 | INFO     | query.querier:get_documents_with_scores:126 - Topscore most similar docs: 0.5404065251350403


[(Document(page_content='Vraag 2 Is bij u bekend in welke gemeenten op dit moment sprake is van de situatie dat meerdere aanbieders bezig zijn met of plannen hebben voor de aanleg van een glasvezelnetwerk? Zo nee, wilt u dit monitoren en deze informatie met de Kamer delen? Antwoord 2 De Autoriteit Consument en Markt (ACM) heeft een interactief dashboard en geografische kaart beschikbaar, waarmee per Nederlands postcodegebied inzichtelijk is hoeveel glasvezelexploitanten in een gebied actief zijn. 2 Dit dashboard en de kaart maken onderdeel uit van de structurele Telecommoni-tor, waarbij de ACM elk kwartaal gegevens opvraagt bij glasvezelexploitanten over het aantal gerealiseerde en geplande glasvezelaansluitingen. Om redenen van bedrijfsgevoeligheid worden daarbij alleen de gerealiseerde aansluitingen gepubliceerd in het dashboard en op de kaart, en niet de geplande aansluitingen.', metadata={'Source': 'https://gegevensmagazijn.tweedekamer.nl/OData/v4/2.0/Document(1b039d7d-7969-4ceb-97

In [6]:
querier.clear_history()